In [2]:
# Sentence segmentation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

text = "This is an example of text that needs to be tokenized. The sent_tokenize function from the nltk library will split the text into sentences."

sentences = sent_tokenize(text)
print(sentences)

['This is an example of text that needs to be tokenized.', 'The sent_tokenize function from the nltk library will split the text into sentences.']


[nltk_data] Downloading package punkt to C:\Users\Sahil
[nltk_data]     Raj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
import numpy as np
emptyarray= np.empty((len(sentences),1,3),dtype=object)
for s in range(len(sentences)):
    emptyarray[s][0][0] = sentences[s]
    emptyarray[s][0][1] = s
emptyarray

array([[['This is an example of text that needs to be tokenized.', 0,
         None]],

       [['The sent_tokenize function from the nltk library will split the text into sentences.',
         1, None]]], dtype=object)

In [26]:
# Tokenization

from string import punctuation
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

In [28]:
# Bi-grams

bi_token=[]
bi_token_length=[]
tri_token_length=[]

for u in range(len(sentences)):
    sent_split1=[w.lower() for w in sentences[u].split(" ")]
    sent_split=[w for w in sent_split1 if w not in stop_words and w not in punctuation and not w.isdigit()]
#     print(sent_split)
    bigrams_list = [bigram for bigram in nltk.bigrams(sent_split)]
#     print(bigrams_list)
    bi_token.append(bigrams_list)
    bi_token_length.append(len(bi_token[u]))

bi_tokens = [(int(o) / max(bi_token_length))*100 for o in bi_token_length]
print(bi_tokens)
print("bitokens feature vector:",(bi_token_length))

[50.0, 100.0]
bitokens feature vector: [3, 6]


In [29]:
# Tri-grams

tri_token=[]
for u in range(len(sentences)):
    sent_split2=[w.lower() for w in sentences[u].split(" ")]
    sent_split3=[w for w in sent_split2 if w not in stop_words and w not in punctuation and not w.isdigit()]
    trigrams_list = [trigram for trigram in nltk.trigrams(sent_split3)]
    tri_token.append(trigrams_list)
    tri_token_length.append(len(tri_token[u]))
tri_tokens = [(int(m) / max(tri_token_length))*100 for m in tri_token_length]
print(tri_tokens)
print("tritokens feature vector:",tri_token_length)

[40.0, 100.0]
tritokens feature vector: [2, 5]


In [33]:
# Sentence Position Feature

import math
def position(l):
    return [index for index, value in enumerate(sentences)]

sent_position= (position(sentences))
print("sentence position:",sent_position)
num_sent=len(sent_position)
print("Total number of sentences:",num_sent)

sentence position: [0, 1]
Total number of sentences: 2


In [34]:
position = []
position_rbm = []

# sentence postion feature of first sentence
sent_pos1_rbm = 1
sent_pos1 = 100
position.append(sent_pos1)
position_rbm.append(sent_pos1_rbm)

# for all sentences except first and last
for x in range(1,num_sent-1):
    s_p= ((num_sent-x)/num_sent)*100
    position.append(s_p)
    s_p_rbm = (num_sent-x)/num_sent
    position_rbm.append(s_p_rbm)
    
# sentence postion feature of last sentence
sent_pos2_rbm = 1
sent_pos2 = 100
position.append(sent_pos2)
position_rbm.append(sent_pos2_rbm)

print("Sentence position feature vector:",position_rbm)

Sentence position feature vector: [1, 1]


In [45]:
# Converting Sentences to Vectors

def convertToVSM(sentences):
    vocabulary = []
    for sents in sentences:
        vocabulary.extend(sents)
    vocabulary = list(set(vocabulary))
    vectors = []
    for sents in sentences:
        vector = []
        for tokenss in vocabulary:
            vector.append(sents.count(tokenss))
        vectors.append(vector)
    return vectors
VSM=convertToVSM(sentences)
print(sentences)
print("SentenceVectors:",VSM)

['This is an example of text that needs to be tokenized.', 'The sent_tokenize function from the nltk library will split the text into sentences.']
SentenceVectors: [[2, 1, 6, 8, 1, 3, 1, 0, 2, 0, 0, 1, 10, 3, 3, 1, 3, 3, 1, 1, 2, 1, 0, 0, 1, 0], [3, 1, 11, 10, 2, 1, 1, 1, 0, 2, 3, 1, 12, 8, 4, 5, 6, 4, 2, 1, 1, 1, 1, 1, 1, 1]]
